In [1]:
from init import *
import time

# Ito processes

- https://en.wikipedia.org/wiki/It%C3%B4_isometry
- https://en.wikipedia.org/wiki/It%C3%B4%27s_lemma

Consider Ito stochastic processes $X(T)$ and $Y(T)$:

\begin{eqnarray}
X(T) &=& X(0) + \int_0^T\mu_1(t)\textrm{d}t + \int_0^T \sigma_{11}(t)\textrm{d}W_{1t} + \int_0^T \sigma_{12}(t)\textrm{d}W_{2t}\\
Y(T) &=& Y(0) + \int_0^T\mu_2(t)\textrm{d}t + \int_0^T \sigma_{21}(t)\textrm{d}W_{1t} + \int_0^T \sigma_{22}(t)\textrm{d}W_{2t}\\
\end{eqnarray}

... or writtent in differntial form:

\begin{eqnarray}
\textrm{d}X_t &=& \mu_1(t) \cdot \textrm{d}t + \sigma_{11}(t) \cdot \textrm{d}W_{1t} + \sigma_{12}(t) \cdot \textrm{d}W_{2t} \\
\textrm{d}Y_t &=& \mu_2(t) \cdot \textrm{d}t + \sigma_{21}(t) \cdot \textrm{d}W_{1t} + \sigma_{22}(t) \cdot \textrm{d}W_{2t} \\
\end{eqnarray}

# Ito isometry

\begin{eqnarray}
\mathbb{E}\left[ \left( \int_0^T X_t \textrm{d}W_t \right)^2 \right] &=&
\mathbb{E}\left[ \int_0^TX_t^2 \textrm{d} t \right] \\
\mathbb{E}\left[ \left( \int_0^T X_t \textrm{d}W_t \right) \cdot \left( \int_0^T Y_t \textrm{d}W_t \right) \right] &=&
\mathbb{E}\left[ \int_0^TX_t Y_t \textrm{d} t \right]
\end{eqnarray}

In a simplified form, the Ito isometry can be formulated as following. For a non-random function $\mu(t)$ the integral
$$
I(T) = \int_0^T \sigma(t) \textrm{d}W_t
$$

for each $T\geq 0$ is normally distributed with expected value zero and variance:

\begin{eqnarray}
\mathbb{E} \left[I(T)\right] &=& 0 \\
\mathbb{E} \left[I^2(T)\right] &=& \int_0^T\sigma^2(t)\textrm{d}t \\
\end{eqnarray}

## Example 1

For example, if we use ordinary (non-stochastic) drift and diffusion functions:
\begin{eqnarray*}
\mu(t) &=& t \\
\sigma(t) &=& t \\
\end{eqnarray*}

With initial condition $X(0)=0$

Then:
\begin{eqnarray*}
\mathbb{E} \left[X(T)\right] &=& \frac{T^2}{2} \\
\mathbb{E} \left[\left(X(T)\right)^2\right] &=& \int_0^T t^2 \textrm{d}t = \frac{T^3}{3}
\end{eqnarray*}

## Using The Monte Carlo Technic

In [2]:
def BrownianMotion_example (
    start        : float,          # X(0)
    drift        : float,
    diffusion    : float,
    time_horizon : float,
    paths        : int    = 100000,
    time_steps   : int    = 1000,
    seed         : int    = 0      # Use -1 to generate the seed number randomly
):
    model = Model()
    model.TimeStart = 0
    model.TimeSteps = time_steps
    model.NumPaths = paths
    model.RandomSeed = seed

    model.Add(IndependentGaussian())
    start = 0
    bm_process = BrownianMotion(start,drift,diffusion)
    model.Add(bm_process) # start, drift, diffusion

    model.evaluations.append(EvaluationPoint(bm_process.Number(),time_horizon))

    reply = requests.post(f'{server}/model',model.json())
    results = EvaluationResults(reply.json())
    
    return results

In [3]:
r = BrownianMotion_example(0,0,1,2)
r.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,2.0,1000,100000,0.001768,0.004486,1.418556,0.010634


In [4]:
class Linear1DInterpolation (Updater):
    def __init__ (self, f, ref:int, xmin:float, xmax:float, points:int=100, title:str=''):
        assert xmin<xmax
        assert points>=2 and points<=1024
        dx = (xmax-xmin)/(points-1)
        Updater.__init__ (
            self,
            name  = 'Linear1DInterpolation',
            start = None,
            refs  = [ref],
            args  = [xmin,xmax,f(xmin)] + [f(xmin+dx*i) for i in range(1,points-1)] + [f(xmax)],
            _title = title
        )

In [5]:
def ItoProcess_example (
    start        : float,          # X(0)
    drift_func,
    diffusion_func,
    time_horizon : float,
    interpolation_points: int = 10,
    paths        : int    = 100000,
    time_steps   : int    = 1000,
    seed         : int    = 0      # Use -1 to generate the seed number randomly
):
    model = Model()
    model.TimeStart = 0
    model.TimeSteps = time_steps
    model.NumPaths = paths
    model.RandomSeed = seed

    model.Add(IndependentGaussian())
    
    ref_time = -1
    drift_process = Linear1DInterpolation(drift_func,ref_time, model.TimeStart,time_horizon,interpolation_points,title='drift')
    model.Add(drift_process)

    diffusion_process = Linear1DInterpolation(drift_func,ref_time, model.TimeStart,time_horizon,interpolation_points,title='diffusion')
    model.Add(diffusion_process)

    start = 0
    X1_process = BrownianMotionRef(start,drift_process.Number(),diffusion_process.Number())
    assert X1_process.refs == [drift_process.Number(),diffusion_process.Number()]
    model.Add(X1_process) # start, drift, diffusion
    
    model.evaluations.append(EvaluationPoint(X1_process.Number(),time_horizon))
    print(model.json())

    reply = requests.post(f'{server}/model',model.json())
    results = EvaluationResults(reply.json())
    
    return results

In [6]:
drift = lambda t: t
diffusion = lambda t: t
results = ItoProcess_example(0,drift,diffusion,time_horizon=2,interpolation_points=2)
results.df()

{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentGaussian"}, {"name": "Linear1DInterpolation", "start": null, "refs": [-1], "args": [0, 2, 0, 2]}, {"name": "Linear1DInterpolation", "start": null, "refs": [-1], "args": [0, 2, 0, 2]}, {"name": "BrownianMotion", "start": 0, "refs": [0, 1]}], "evaluations": [{"state": 2, "time": 2, "histograms": []}], "RandomSeed": 0, "RunTimeoutSeconds": 1, "MemoryLimitKB": 1}


,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,Linear1DInterpolation,,0,2.0,1000,100000,2.000000,0.000000,0.000000,NaN
1,Linear1DInterpolation,,1,2.0,1000,100000,2.000000,0.000000,0.000000,NaN
2,BrownianMotion,,2,2.0,1000,100000,2.001822,0.005185,1.639506,-0.003944


## Is this result correct?

The process is:

$$
X(T) = \int_0^2t\cdot \textrm{d}t + \int_0^2 t\cdot\textrm{d}W_t
$$

Expected value (from the drift term):
$$\mathbb{E}\left[X(2)\right] = \int_0^2 t \textrm{d}t =  \frac{t^2}{2} \bigg\rvert_0^2 = 2$$ 
Variance (from the diffusion term):
$$\textrm{Var}\left[X(2)\right] =  \int_0^2 t^2 \textrm{d}t =  \frac{t^3}{3} \bigg\rvert_0^2 = \frac{8}{3}$$ 
And stddev is a squre root of the variance:
$$\sqrt{\frac{8}{3}} = 1.632993161855452$$

Yes, the numbers do match!

# Itô's lemma

\begin{eqnarray}
\textrm{d}X_t &=& \mu_t \textrm{d}t + \sigma_t\textrm{d}W_t
\\
Y_t &=& f(t,X_t)
\end{eqnarray}

\begin{eqnarray}
\textrm{d} f(t,X_t) &=& 
\left(
\frac{\partial f}{\partial t} +
\mu_t \frac{\partial f}{\partial x} +
\frac{1}{2}\sigma_t^2\frac{\partial^2 f}{\partial x^2}
\right)\textrm{d} t
+
\sigma_t \frac{\partial f}{\partial x}\textrm{d}W_t
\end{eqnarray}

Stochastic calculus rules for differentials of the second order:
\begin{eqnarray}
\textrm{d}t \cdot \textrm{d}t &=& 0
\\
\textrm{d}t \cdot \textrm{d}W_t &=& 0
\\
\textrm{d}W_t \cdot \textrm{d}W_t &=& \textrm{d}t
\end{eqnarray}

# Geometrical brownian motion

\begin{eqnarray}
f(t,x) &=& e^x
\\
\frac{\partial f}{\partial t} &=& 0
\\
\frac{\partial f}{\partial x} &=& e^x
\\
\frac{\partial^2 f}{\partial x^2} &=& e^x
\\
\textrm{d}\exp\left(X_t\right) &=& \exp(X_t) \left[ \left(\mu_t + \frac{\sigma_t^2}{2}\right)\textrm{d}t + \sigma_t \textrm{d}W_t\right]
\end{eqnarray}

\begin{cases}
\mu(t) &=& \cdots
\\
\sigma(t) &=& \cdots
\\
\textrm{d}S(t) &=& \mu(t) S(t) \textrm{d}t + \sigma(t) S(t) \textrm{d}W_t
\end{cases}